## Felles import for alle kodeblokker

In [1]:
import sys
sys.path.append("../analysis")
from analysis import DataAnalyzer
analyzer = DataAnalyzer("../../data/processed")

import plotly.graph_objects as go
from dash import Dash, html, dcc, Input, Output
import calendar

import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

## Interaktiv analyse med dash app

In [2]:
cities = ["oslo", "tromso"]
element_ids = [
    "sum(precipitation_amount P1D)",
    "mean(air_temperature P1D)",
    "mean(wind_speed P1D)",
    "max(air_temperature P1D)",
    "min(air_temperature P1D)",
    "range(air_temperature P1D)"
]

app = Dash(__name__)
app.title = "Klimatologi med/uten outliers"

app.layout = html.Div(
    style={"maxWidth": "900px", "margin": "auto"},
    children=[
        html.H1("Klimatologisk Månedsmiddel"),

        html.Label("Måletype (elementId):"),
        dcc.Dropdown(
            id="element-dropdown",
            options=[{"label": e, "value": e} for e in element_ids],
            value=element_ids[0],
            clearable=False,
        ),

        html.Label("Statistikk:"),
        dcc.RadioItems(
            id="stat-radio",
            options=[
                {"label": "Gjennomsnitt", "value": "mean"},
                {"label": "Median", "value": "median"},
                {"label": "Std.avvik", "value": "std"},
            ],
            value="mean",
            inline=True,
        ),

        html.Label("Outliers:"),
        dcc.RadioItems(
            id="outlier-radio",
            options=[
                {"label": "Ta med", "value": "med"},
                {"label": "Uten", "value": "uten"},
            ],
            value="med",
            inline=True,
        ),

        html.Label("By(er):"),
        dcc.Checklist(
            id="city-checklist",
            options=[{"label": c.capitalize(), "value": c} for c in cities],
            value=cities,
            inline=True,
        ),

        dcc.Graph(id="climate-graph"),
    ],
)

@app.callback(
    Output("climate-graph", "figure"),
    Input("element-dropdown", "value"),
    Input("stat-radio", "value"),
    Input("outlier-radio", "value"),
    Input("city-checklist", "value"),
)
def update_graph(element_id, statistikk, outlier_choice, selected_cities):

    fig = go.Figure()
    month_order = [calendar.month_abbr[m].capitalize() for m in range(1, 13)]
    remove = (outlier_choice == "uten")

    for city in selected_cities:
        df_klima = analyzer.langtidsmiddel_per_måned(
            by=city,
            element_id=element_id,
            remove_outliers=remove,
            statistikk=statistikk,
        )
        df_klima = df_klima.sort_values("month")

        fig.add_trace(
            go.Bar(
                x=df_klima["month_name"],
                y=df_klima["verdi"],
                name=city.capitalize(),
            )
        )

    t_out = "uten outliers" if remove else "med outliers"
    fig.update_layout(
        title=f"Klimatologisk {statistikk} per måned – {element_id} ({t_out})",
        xaxis_title="Måned",
        yaxis_title="Verdi",
        barmode="group",
        template="plotly_white",
        xaxis=dict(categoryorder="array", categoryarray=month_order),
    )
    return fig


if __name__ == "__main__":
    app.run(debug=True, port=8051)

### Sjekke data for bestemt dag basert på historikk. 

In [ ]:

dag_slider = widgets.IntSlider(value=17, min=1, max=31, description='Dag:')
maaned_slider = widgets.IntSlider(value=5, min=1, max=12, description='Måned:')
by_dropdown = widgets.SelectMultiple(
    options=['oslo', 'tromso'],
    value=['oslo'],
    description='By(er):'
)

element_dropdown = widgets.Dropdown(
    options=[
        'sum(precipitation_amount P1D)',
        'max(air_temperature P1D)',
        'mean(wind_speed P1D)'
    ],
    value='sum(precipitation_amount P1D)',
    description='Element:'
)
time_offset_dropdown = widgets.Dropdown(
    options=['PT0H', 'PT6H', 'PT12H', 'PT18H'],
    value='PT0H',
    description='Time Offset:'
)
knapp = widgets.Button(description="Vis statistikk")
output = widgets.Output()
def vis_statistikk(b):
    with output:
        clear_output()
        dag = dag_slider.value
        maaned = maaned_slider.value
        valgte_byer = by_dropdown.value
        element_id = element_dropdown.value
        time_offset = time_offset_dropdown.value
        by_visningsnavn = {'oslo': 'Oslo', 'tromso': 'Tromsø'}
        stats_data = {}
        
        for by in valgte_byer:
            try:
                df = analyzer.stats._load_city(by)
                df_filtered = df[
                    (df['elementId'] == element_id) &
                    (df['timeOffset'] == time_offset)
                ].copy()

                df_filtered['referenceTime'] = pd.to_datetime(df_filtered['referenceTime'], utc=True)
                df_filtered = df_filtered[
                    (df_filtered['referenceTime'].dt.month == maaned) & 
                    (df_filtered['referenceTime'].dt.day == dag)
                ]
                
                df_filtered['value'] = pd.to_numeric(df_filtered['value'], errors='coerce')
                
                if not df_filtered.empty:
                    stats_data[by_visningsnavn[by]] = df_filtered['value'].mean()
            except Exception as e:
                print(f"Feil ved behandling av data for {by}: {e}")
        if not stats_data:
            print("Ingen data tilgjengelig for valgt dag/by.")
            return
        
        plt.figure(figsize=(10, 5))
        plt.bar(stats_data.keys(), stats_data.values(), edgecolor='black')
        plt.title(f"Gjennomsnittlig {element_id} for {dag:02d}.{maaned:02d}")
        plt.ylabel('Verdi')
        plt.grid(axis='y')
        plt.tight_layout()
        plt.show()
        
        print("\nGjennomsnittsverdier:")
        for by, verdi in stats_data.items():
            print(f"{by}: {verdi:.2f}")
        
        print("\nOutlier-informasjon:")
        for by_kode, by_navn in zip(valgte_byer, [by_visningsnavn[b] for b in valgte_byer]):
            try:
                df = analyzer.stats._load_city(by_kode)
                df_filtered = df[
                    (df['elementId'] == element_id) &
                    (df['timeOffset'] == time_offset)
                ].copy()
                
                df_filtered['value'] = pd.to_numeric(df_filtered['value'], errors='coerce')
                
                has_outliers = analyzer.detector.detect_iqr(df_filtered['value'])
                outlier_count = has_outliers.sum()
                
                if outlier_count > 0:
                    print(f"{by_navn}: {outlier_count} outliers funnet")
                else:
                    print(f"{by_navn}: Ingen outliers funnet")
            except Exception as e:
                print(f"Kunne ikke analysere outliers for {by_navn}: {e}")
knapp.on_click(vis_statistikk)
display(maaned_slider, dag_slider, by_dropdown, element_dropdown, time_offset_dropdown, knapp, output)

IntSlider(value=5, description='Måned:', max=12, min=1)

IntSlider(value=17, description='Dag:', max=31, min=1)

SelectMultiple(description='By(er):', index=(0,), options=('oslo', 'tromso'), value=('oslo',))

Dropdown(description='Element:', options=('sum(precipitation_amount P1D)', 'max(air_temperature P1D)', 'mean(w…

Dropdown(description='Time Offset:', options=('PT0H', 'PT6H', 'PT12H', 'PT18H'), value='PT0H')

Button(description='Vis statistikk', style=ButtonStyle())

Output()